clean
cnn: 0.9970
abc: 0.9971

fgsm
cnn: 0.2797
abc: 0.5499

pgd20
cnn: 0.9813
abc: 0.9808

foolbox:
pgd20 alpha=0.1 epsilon=0.1
cnn: 0.0242
abc: 0.2902

fgsm epsilon=0.1
cnn: 0.1504
abc: 0.8913

In [1]:
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.tools import dotdict
from driver.driver import ABC_Driver
torch.cuda.is_available()
torch.cuda.set_device(0)

In [2]:
mnist_args = dotdict()

mnist_args.name = 'mnist'
# mnist_args.device = ['cuda:4', 'cuda:5']
mnist_args.device = ['cuda:0']
mnist_args.train_batch_size = 120
mnist_args.predict_batch_size = 120

mnist_args.train_epochs = 25
mnist_args.lr = 0.2
mnist_args.criterion = 'CE'
mnist_args.optimizer = 'SGD'
mnist_args.scheduler = 'multistep3'
mnist_args.attack = {'deepfool':(0.1,0.1,20),'apgd-ce':(0.1,)}

activation = 'relu'
input_channel = 1
# knpp = [30,60,120,180,240,300]

# mnist_args.layers=[
#     ('cnn2d', ((input_channel, knpp[0], (7,7), 1, 3, 1, 1), 1, None, None, activation, False)),
#     ('atc2d', ((knpp[0], knpp[1], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
#     ('atc2d', ((knpp[1], knpp[2], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('atc2d', ((knpp[2], knpp[3], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, True)),
#     ('atc2d', ((knpp[3], knpp[4], (3,3), 1, 1, 1, knpp[0]), 1, 'first', (2,2), activation, True)),
#     ('cnn2d', ((knpp[4], knpp[5], (3,3), 1, 1, 1, knpp[0]), 1, None, None, activation, False)),
#     ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 0, 1, knpp[0]), 1, None, None, False, False)),
#     ('adptavgpool', (1,1)),
#     ('linear', (knpp[-1], 10, (1,2,3)))
# ]

# mnist_args.layers=[
#     ('cnn2d', ((1, 24, (7, 7), 1, 3, 1, 1), 1, None, None, 'relu', False)), 
#     ('atc2d', ((24, 48, (3, 3), 1, 1, 1, 24), 1, None, None, 'relu', True)), 
#     ('atc2d', ((48, 96, (3, 3), 1, 1, 1, 24), 1, 'first', (2, 2), 'relu', True)), 
#     ('atc2d', ((96, 144, (3, 3), 1, 1, 1, 24), 1, None, None, 'relu', True)), 
#     ('atc2d', ((144, 192, (3, 3), 1, 1, 1, 24), 1, 'first', (2, 2), 'relu', True)), 
#     ('cnn2d', ((192, 240, (3, 3), 1, 1, 1, 24), 1, None, None, 'relu', False)), 
#     ('cnn2d', ((240, 240, (3, 3), 1, 0, 1, 24), 1, None, None, False, False)), 
#     ('adptavgpool', (1, 1)), 
#     ('linear', (240, 10, (1, 2, 3)))
# ]


# mnist_args.layers=[
#     ('cnn2d', ((1, 24, (7, 7), 1, 3, 1, 1), 1, None, None, 'relu', False)), 
#     ('cnn2d', ((24, 48, (3, 3), 1, 1, 1, 1), 2, None, None, 'relu', True)), 
#     ('cnn2d', ((48, 96, (3, 3), 1, 1, 1, 1), 2, 'first', (2, 2), 'relu', True)), 
#     ('cnn2d', ((96, 144, (3, 3), 1, 1, 1, 1), 2, None, None, 'relu', True)), 
#     ('cnn2d', ((144, 192, (3, 3), 1, 1, 1, 1), 2, 'first', (2, 2), 'relu', True)), 
#     ('cnn2d', ((192, 240, (3, 3), 1, 1, 1, 1), 1, None, None, 'relu', False)), 
#     ('cnn2d', ((240, 240, (3, 3), 1, 0, 1, 1), 1, None, None, False, False)), 
#     ('adptavgpool', (1, 1)), 
#     ('linear', (240, 10, (1, 2, 3)))
# ]


# mnist_args.layers=[
#     ('cnn2d', ((1, 24, (7, 7), 1, 3, 1, 1), 1, None, None, 'relu', False)), 
#     ('atrc2d', ((24, 48, (3, 3), 1, 1, 1, 24), 1, None, None, 'relu', True)), 
#     ('atrc2d', ((48, 96, (3, 3), 1, 1, 1, 24), 1, 'first', (2, 2), 'relu', True)), 
#     ('atrc2d', ((96, 144, (3, 3), 1, 1, 1, 24), 1, None, None, 'relu', True)), 
#     ('atrc2d', ((144, 192, (3, 3), 1, 1, 1, 24), 1, 'first', (2, 2), 'relu', True)), 
#     ('cnn2d', ((192, 192, (3, 3), 1, 1, 1, 24), 1, None, None, 'relu', False)), 
#     ('cnn2d', ((192, 192, (3, 3), 1, 0, 1, 24), 1, None, None, False, False)), 
#     ('adptavgpool', (1, 1)), 
#     ('linear', (192, 10, (1, 2, 3)))
# ]



# knpp = [24,48,96,192]
knpp = [30,60,120,240]

mnist_args.layers=[
    ('cnn2d', ((input_channel, knpp[0], (3,3), 1, 1, 1, 1), 1, None, None, activation, False)),
    ('atrc2d', ((knpp[0], knpp[1], (3,3), 1, 1, 1, 1), 1, None, None, activation, True)),
    ('atrc2d', ((knpp[1], knpp[2], (3,3), 1, 1, 1, 1), 1, 'first', (2,2), activation, True)),
    ('atrc2d', ((knpp[2], knpp[3], (3,3), 1, 1, 1, 1), 1, 'first', (2,2), activation, True)),
    ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 0, 1, 1), 1, None, None, activation, False)),
    ('cnn2d', ((knpp[-1], knpp[-1], (3,3), 1, 0, 1, 1), 1, None, None, False, False)),
    ('adptavgpool', (1,1)),
    ('linear', (knpp[-1], 10, (1,2,3)))
]

In [3]:
driver = ABC_Driver(mnist_args, None, record_path=None, if_hash=False)
# driver.train()

Use: ['cuda:0']
add record: 05/09/2023 06:51


In [4]:
# torch.save(driver.model.state_dict(), "save/MNIST_CNN_2023_03_18.pt")
driver.model.load_state_dict(torch.load("save/MNIST_ARC_2023_03_14.pt"))
# driver.model.load_state_dict(torch.load("save/MNIST_CNN_2023_02_27.pt"))
# driver.model.load_state_dict(torch.load("save/MNIST_ABC_2023_02_26.pt"))

<All keys matched successfully>

In [5]:
driver.metric(test_attack=True)

setting parameters for standard version
using standard version including apgd-ce.


/home/fsuser/Attention_based_CNN/Mark_Exp/attack/attack.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = torch.tensor(self.data_loader.origin_data)
/home/fsuser/Attention_based_CNN/Mark_Exp/attack/attack.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(self.data_loader.dataset.targets)


initial accuracy: 99.67%
apgd-ce - 1/100 - 90 out of 100 successfully perturbed
apgd-ce - 2/100 - 83 out of 100 successfully perturbed
apgd-ce - 3/100 - 89 out of 100 successfully perturbed
apgd-ce - 4/100 - 92 out of 100 successfully perturbed
apgd-ce - 5/100 - 81 out of 100 successfully perturbed
apgd-ce - 6/100 - 80 out of 100 successfully perturbed
apgd-ce - 7/100 - 90 out of 100 successfully perturbed
apgd-ce - 8/100 - 90 out of 100 successfully perturbed
apgd-ce - 9/100 - 84 out of 100 successfully perturbed
apgd-ce - 10/100 - 91 out of 100 successfully perturbed
apgd-ce - 11/100 - 89 out of 100 successfully perturbed
apgd-ce - 12/100 - 88 out of 100 successfully perturbed
apgd-ce - 13/100 - 87 out of 100 successfully perturbed
apgd-ce - 14/100 - 86 out of 100 successfully perturbed
apgd-ce - 15/100 - 85 out of 100 successfully perturbed
apgd-ce - 16/100 - 90 out of 100 successfully perturbed
apgd-ce - 17/100 - 87 out of 100 successfully perturbed
apgd-ce - 18/100 - 93 out of 100

clean      0.9967
apgd-ce    0.1490
dtype: float64

In [5]:
driver.metric(test_attack=True)

setting parameters for standard version
using standard version including apgd-ce.
initial accuracy: 99.70%
apgd-ce - 1/100 - 99 out of 100 successfully perturbed
apgd-ce - 2/100 - 100 out of 100 successfully perturbed
apgd-ce - 3/100 - 100 out of 100 successfully perturbed
apgd-ce - 4/100 - 99 out of 100 successfully perturbed
apgd-ce - 5/100 - 99 out of 100 successfully perturbed
apgd-ce - 6/100 - 98 out of 100 successfully perturbed
apgd-ce - 7/100 - 100 out of 100 successfully perturbed
apgd-ce - 8/100 - 97 out of 100 successfully perturbed
apgd-ce - 9/100 - 97 out of 100 successfully perturbed
apgd-ce - 10/100 - 100 out of 100 successfully perturbed
apgd-ce - 11/100 - 99 out of 100 successfully perturbed
apgd-ce - 12/100 - 99 out of 100 successfully perturbed
apgd-ce - 13/100 - 99 out of 100 successfully perturbed
apgd-ce - 14/100 - 99 out of 100 successfully perturbed
apgd-ce - 15/100 - 97 out of 100 successfully perturbed
apgd-ce - 16/100 - 100 out of 100 successfully perturbed
a

clean       0.9970
deepfool    0.1966
apgd-ce     0.0095
dtype: float64